In [1]:
import custom_models
#python packages
from PIL import Image
from tqdm.notebook import tqdm
#from tqdm import tqdm
import gc
import datetime
import os
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from skimage import io
#torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
#torchvision
import torchvision
from torchvision import datasets, models, transforms
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using GPU!")
else:
    print("WARNING: Could not find GPU! Using CPU only")

PyTorch Version:  1.2.0
Torchvision Version:  0.4.0a0+6b959ee
Using GPU!


In [2]:
def make_CNN(model_name, num_classes, resume_from = None):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    # The model (nn.Module) to return
    model_ft = None
    # The input image is expected to be (input_size, input_size)
    input_size = 0
    
    # You may NOT use pretrained models!! 
    use_pretrained = False
    
    # By default, all parameters will be trained (useful when you're starting from scratch)
    # Within this function you can set .requires_grad = False for various parameters, if you
    # don't want to learn them

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224
        
    elif model_name == "vgg13":
        """ VGG13_bn
        """
        model_ft = custom_models.vgg13_bn(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224
        
    elif model_name == "vgg16":
        """ VGG16_bn
        """
        model_ft = custom_models.vgg13_bn(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224
        
    elif model_name == "vgg19":
        """ VGG19_bn
        """
        model_ft = custom_models.vgg13_bn(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes) 
        input_size = 224

    else:
        raise Exception("Invalid model name!")
    
    return model_ft, input_size

In [3]:
class Chest_Disease_Net(nn.Module):
    """
    fc1: number of neurons in the hidden fully connected layer
    """
    def __init__(self, cnn_model_name, num_classes, num_multimodal_features=12, fc1_out=32, resume_from=None):
        #num_classes = 14
        #num_multimodal_features= 12
        super(Chest_Disease_Net, self).__init__()
        self.cnn, self.input_size = make_CNN(cnn_model_name, num_classes)#models.vgg11(pretrained=False, progress = True)
        #define output layers
        self.fc1 = nn.Linear(num_classes + num_multimodal_features, fc1_out) #takes in input of CNN and multimodal input
        self.fc2 = nn.Linear(fc1_out, num_classes)
        if resume_from is not None:
            print("Loading weights from %s" % resume_from)
            self.load_state_dict(torch.load(resume_from))
        
    def forward(self, image, data):
        x1 = self.cnn(image)
        #print("x1", x1.shape)
        x2 = data
        #print("x2", x2.shape)
        #print("x1: ", x1, type(x1))
        #print("x2: ", x2, type(x2))
        #x = torch.cat((x1, x2), dim=1)  
        x = torch.cat((x1.float(), x2.float()), dim=1) ### ???
        #print("concat", x.shape)
        x = F.relu(self.fc1(x))
        #print("relu", x.shape)
        x = self.fc2(x)
        print('forward output: ', x)
       # print("fc2", x.shape)
        return x.double() ### ???

In [4]:
class MultimodalDataset(Dataset):
    """
    Custom dataset definition
    """
    def __init__(self, pandas_dataframe, img_path, transform=None):
        """
        """
        self.df = pandas_dataframe
        self.img_path = img_path
        self.transform = transform
        self.diseases = self.get_diseases()
        
            
    def __getitem__(self, index):
        """
        """
        img_name = self.df.iloc[index]["img_name"] 
        img_path = os.path.join(self.img_path, img_name)
        image = Image.open(img_path)
        image = image.convert("RGB")
        image = np.asarray(image)
        if self.transform is not None:
            image = self.transform(image)
        dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor # ???
        features = np.fromstring(self.df.iloc[index]["feature"][1:-1], sep=",") # ???
        features = torch.from_numpy(features.astype("float")) # ???
        #label = int(self.df.iloc[index]['label'])
        labels = torch.tensor(list(self.df.iloc[index][self.diseases]), dtype = torch.float64)
        #print("Label type: ", type(label))
        #label = np.int_(label) #???
        #print("label type post casting: ", type(label))
        return image, features, labels
        
    def __len__(self):
        return len(self.df)
    
    def get_diseases(self):
        cols = list(self.df.columns)
        cols.remove('feature')
        cols.remove('img_name')
        return cols
        

In [5]:
def get_dataloaders(provided_df, input_size, batch_size):
    # How to transform the image when you are loading them.
    # you'll likely want to mess with the transforms on the training set.
    
    # For now, we resize/crop the image to the correct input size for our network,
    # then convert it to a [C,H,W] tensor, then normalize it to values with a given mean/stdev. These normalization constants
    # are derived from aggregating lots of data and happen to produce better results.
    data_transforms = {
        'test': transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.225])
        ])
    }
    # Create training and validation datasets
    data_set = MultimodalDataset(pandas_dataframe = provided_df, 
                                         img_path="/storage/images", 
                                         transform= data_transforms["test"])
    dataloaders_dict = DataLoader(data_set, batch_size=batch_size, shuffle=False, num_workers=4)
    return dataloaders_dict

In [6]:
def load_model(multimodal, model_name, path, num_classes):   
    
    if multimodal:
        model = Chest_Disease_Net(cnn_model_name = model_name, num_classes = num_classes, resume_from = None)
        input_size = model.input_size
    else:
        model, input_size = make_CNN(model_name=model_name,num_classes=num_classes, resume_from = None)

    model_path = path
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint)
    return model

In [7]:
df = pd.read_csv("./data/test_dataset8.csv").drop(['dataset_type', 'disease'],axis=1)

In [9]:
def per_disease9(binary, multimodal, model_name, path_to_model, num_classes=8): 
    #load test_dataset
    df = pd.read_csv("./data/test_dataset{}.csv".format(num_classes)).drop(['dataset_type', 'disease'],axis=1)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(df)
    disease_names = ['cardiomegaly', 'nodule', 'pneumothorax', 'effusion', 
                     'pneumonia', 'infiltration', 'atelectasis','mass']
        
    disease_to_label = {}
    for index,value in enumerate(disease_names):
        disease_to_label[index] = value
      
    input_size = 256
    batch_size = 64
    
    
    model = load_model(multimodal, model_name, path_to_model, 8)    
    model.to(device)
    model.eval()

    for i in range(8):
        new_df = df[['feature','img_name',disease_names[i]]]
        new_df = new_df.loc[new_df[disease_names[i]] == binary]
        dataloader = get_dataloaders(new_df, input_size, batch_size)
        correct = 0
        test_total = 0

        with torch.no_grad():
            for data in tqdm(dataloader):
                images, features, labels = data
                
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                
#                 _, predicted = torch.max(outputs.data, 1)
                
                            
                predicted = (outputs > 0).type(torch.float64)
                predicted = predicted [0 : batch_size , i:i+1]
        
                       
                test_total += labels.size(0)

                correct += (predicted == labels).sum().item()
        if not binary:
            print("False Positive")
        print('Accuracy of the network on test images for ' + disease_names[i] + ': %d %%' % (
                100 * (correct / test_total)))

        


In [10]:
predicted = per_disease9(1, False, "alexnet", "/home/ubuntu/6.867-xray-project/weights/alexnet_best_weights_1.pt")


Accuracy of the network on test images for cardiomegaly: 71 %



Accuracy of the network on test images for nodule: 21 %



Accuracy of the network on test images for pneumothorax: 35 %



Accuracy of the network on test images for effusion: 70 %



Accuracy of the network on test images for pneumonia: 55 %



Accuracy of the network on test images for infiltration: 52 %



Accuracy of the network on test images for atelectasis: 74 %



Accuracy of the network on test images for nofinding: 28 %



Accuracy of the network on test images for mass: 0 %


In [11]:
predicted = per_disease9(0, False, "alexnet", "/home/ubuntu/6.867-xray-project/weights/alexnet_best_weights_1.pt")


False Positive
Accuracy of the network on test images for cardiomegaly: 71 %



False Positive
Accuracy of the network on test images for nodule: 84 %



False Positive
Accuracy of the network on test images for pneumothorax: 84 %



False Positive
Accuracy of the network on test images for effusion: 70 %



False Positive
Accuracy of the network on test images for pneumonia: 70 %



False Positive
Accuracy of the network on test images for infiltration: 71 %



False Positive
Accuracy of the network on test images for atelectasis: 54 %



False Positive
Accuracy of the network on test images for nofinding: 45 %



False Positive
Accuracy of the network on test images for mass: 0 %
